# DSPy on Azure OpenAI - 241026

In [ ]:
# Test AzureOpenAI endpoints 
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

deployment='gpt-4o-eastus-0806'

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
    )
        
# Send a completion call to generate an answer
completion = client.chat.completions.create(
    model=deployment,
    messages = [
        {
        "role": "system",
        "content": "You are an MIT PhD in Physics, specializing in quantum physics."
        },
        {
        "role": "user",
        "content": "What is a black hole?"
        }
    ]
    # max_tokens=4096
)

#print(completion.model_dump_json(indent=2))
content = completion.choices[0].message.content
print(content)
print(len(content))

In [ ]:
import dspy

lm = dspy.LM('openai/gpt-4o-mini-eastus-0718')
dspy.configure(lm=lm)

qa = dspy.Predict('question: str -> response: str')
qa(question="what are high memory and low memory on linux?").response


In [20]:
from dotenv import dotenv_values, load_dotenv

load_dotenv()

config=dotenv_values(".env")
azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_EASTUS_API_KEY")
embedding_model="text-embedding-3-small-eastus"
deployment='gpt-4o-eastus-0806'

import dspy

turbo = dspy.AzureOpenAI(
    api_key=api_key,
    api_version= "2024-06-01",
    api_base=azure_endpoint,
    model=deployment,
)

dspy.configure(lm=turbo)




In [22]:
qa = dspy.Predict('question: str -> response: str')
qa(question="what are high memory and low memory on linux?").response

"High memory and low memory in Linux refer to different regions of the system's RAM. Low memory is directly accessible by the kernel and is typically the first 896 MB on a 32-bit system. High memory is the rest of the RAM that requires special handling to be accessed by the kernel. This distinction is more relevant in 32-bit systems due to their limited address space. In 64-bit systems, the distinction is less significant because they can address much larger amounts of memory directly."